## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-22-17-53-04 +0000,nypost,Kimmel’s ‘Cousin Sal’ reveals ‘more bombshells...,https://nypost.com/2025/09/22/us-news/kimmels-...
1,2025-09-22-17-50-23 +0000,nypost,Nvidia investing up to $100B in ChatGPT owner ...,https://nypost.com/2025/09/22/business/nvidia-...
2,2025-09-22-17-47-30 +0000,bbc,Lib Dems urge probe into Trump golf course lob...,https://www.bbc.com/news/articles/crkjlx5lr5po...
3,2025-09-22-17-43-44 +0000,startribune,"Souhan: After a historic season, why not MVPhee?",https://www.startribune.com/wnba-mvp-collier-l...
4,2025-09-22-17-40-57 +0000,bbc,Air India crash aftermath handled 'irresponsib...,https://www.bbc.com/news/articles/crmex7912nro...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,51
53,kirk,33
52,charlie,25
108,memorial,17
57,he,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
221,2025-09-21-22-51-23 +0000,nypost,Trump calls Charlie Kirk one of ‘America’s gre...,https://nypost.com/2025/09/21/us-news/trump-ca...,153
129,2025-09-22-11-14-50 +0000,nypost,Charlie Kirk inspired Trump and Elon Musk to r...,https://nypost.com/2025/09/22/us-news/charlie-...,151
213,2025-09-21-23-30-00 +0000,wsj,President Trump and the upper echelon of his a...,https://www.wsj.com/us-news/charlie-kirk-funer...,149
184,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...,147
63,2025-09-22-15-26-06 +0000,nypost,The 5 most emotional moments of the Charlie Ki...,https://nypost.com/2025/09/22/us-news/the-five...,144


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
221,153,2025-09-21-22-51-23 +0000,nypost,Trump calls Charlie Kirk one of ‘America’s gre...,https://nypost.com/2025/09/21/us-news/trump-ca...
222,41,2025-09-21-22-27-41 +0000,nypost,Erika Kirk forgives assassin who killed husban...,https://nypost.com/2025/09/21/us-news/erika-ki...
175,41,2025-09-22-06-35-15 +0000,bbc,South Korea would accept a Trump-Kim deal to f...,https://www.bbc.com/news/articles/cwy91w0e1z2o...
58,39,2025-09-22-15-36-02 +0000,cbc,"Ford government lays out demands for bail, sen...",https://www.cbc.ca/news/canada/toronto/ontario...
60,38,2025-09-22-15-31-23 +0000,nypost,Howard Stern blasts ABC suspension of Jimmy Ki...,https://nypost.com/2025/09/22/media/howard-ste...
184,38,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...
8,35,2025-09-22-17-29-04 +0000,nypost,Jill Biden’s ex-spokesman reveals ‘cult’ of ‘m...,https://nypost.com/2025/09/22/us-news/michael-...
30,31,2025-09-22-16-47-01 +0000,nyt,American Investors Will License and Oversee Ti...,https://www.nytimes.com/2025/09/22/us/politics...
11,31,2025-09-22-17-21-27 +0000,nypost,Zohran Mamdani slammed by high-profile NY rabb...,https://nypost.com/2025/09/22/us-news/zohran-m...
65,30,2025-09-22-15-22-13 +0000,cbc,India says trade talks with Canada have resume...,https://www.cbc.ca/news/politics/canada-india-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
